In [4]:
import xml.etree.ElementTree as ET
import numpy as np
import json

In [41]:
class Data():
    def __init__(self, P, num):
        root_node = ET.parse('Datas/InstanceP='+P+'/Instance'+num+'.xml').getroot()
        nb_pair=0
        nb_donor=0
        id_max=0
        
        #Calcul du nombre de pair, du nombre de donneur et de id_max
        for donor in root_node.findall('entry'):
            nb_donor=nb_donor+1
            id=donor.attrib['donor_id']
            if(int(id) > int(id_max)):
                id_max=donor.attrib['donor_id']
            if(donor.find('altruistic')==None):
                nb_pair=nb_pair+1
        print('nombre de pair : ',  nb_pair, ' ; nombre de donneur : ', nb_donor)
        print('id max : ', id_max)
        print()
        #Autre méthode utilisant le fichier json (ne fonctionne puisque le nombre de donneurn'est pas égale à nombre de patients x 
            # x110%)
        #with open('Datas/InstanceP='+str(50)+'/config.json') as json_data:
            #data_dict = json.load(json_data)
            #nb_paire=data_dict['patientsPerInstance']
            #nb_donor= int(nb_paire*1.12)
        
        #initialisation tableau des couts des échanges, tableau des donneurs altruistes et des paires
        cost = np.zeros((int(id_max)+1,int(id_max)+1))
        altruist = np.zeros((int(id_max)+1,int(id_max)+1))
        pair = np.zeros((int(id_max)+1,int(id_max)+1))
        listDonor=[]
        
        #parcous du fichier html et remplissage des tableaux
        for donor in root_node.findall('entry'):
            #### Get the value from the attribute 'donor_id'###
            id = donor.attrib['donor_id']
            listDonor.append(id)
            print('id du donneur :',int(id))
            for exchange in donor.findall('matches/match'):
                patient=exchange.find('recipient')
                score=exchange.find('score')
                if(patient.text!=None):
                    cost[int(id)][int(patient.text)]=int(score.text)
                #print('patient : '+ patient.text +' avec un score de ' + score.text)
                if(donor.find('altruistic')!=None):
                    altruist[int(id),int(patient.text)]=1
                else : 
                    pair[int(id),int(patient.text)]=1
                #print('le donneur est altruiste : ', altruist)
        print(listDonor)
        self.nb_pair=nb_pair
        self.nb_donor=nb_donor
        self.cost=cost
        self.altruist=altruist
        self.pair=pair
        self.listDonor=listDonor

In [42]:
Data(str(50),str(0))

nombre de pair :  50  ; nombre de donneur :  56
id max :  127

id du donneur : 0
id du donneur : 1
id du donneur : 2
id du donneur : 9
id du donneur : 10
id du donneur : 11
id du donneur : 19
id du donneur : 20
id du donneur : 21
id du donneur : 23
id du donneur : 24
id du donneur : 26
id du donneur : 35
id du donneur : 38
id du donneur : 40
id du donneur : 44
id du donneur : 45
id du donneur : 46
id du donneur : 47
id du donneur : 50
id du donneur : 51
id du donneur : 52
id du donneur : 53
id du donneur : 54
id du donneur : 57
id du donneur : 62
id du donneur : 64
id du donneur : 67
id du donneur : 68
id du donneur : 70
id du donneur : 71
id du donneur : 74
id du donneur : 75
id du donneur : 85
id du donneur : 86
id du donneur : 90
id du donneur : 97
id du donneur : 100
id du donneur : 101
id du donneur : 102
id du donneur : 104
id du donneur : 108
id du donneur : 110
id du donneur : 111
id du donneur : 113
id du donneur : 114
id du donneur : 115
id du donneur : 119
id du donneur : 12